<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/05_CHIRPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract monthly rainfall data

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
# aoi_name = "MKR_NS_buff_5km"

aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR")
aoi_name = "MKR"

# GEE layer ID
layer_name = "RS_012"

# Date parameters
start_year = 2012
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

## CHIRPS Daily: Climate Hazards Group InfraRed Precipitation With Station Data (Version 2.0 Final)

Spatial resolution is 5km so not sure if Matt is going to find this uesful - check first.


In [ ]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

## Rainfall processing - annual sum

In [ ]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
years = ee.List.sequence(start_year, end_year)

def yearly_rain(focal_year):
  start = ee.Date.fromYMD(focal_year, 1, 1)
  end = ee.Date.fromYMD(focal_year, 12, 31)
  date_range = ee.DateRange(start, end)
  return chirps \
        .filterDate(date_range) \
        .sum() \
        .clip(aoi)

rain_years = ee.ImageCollection.fromImages(years.map(yearly_rain).flatten())
print(rain_years.getInfo())
rain_bands = rain_years.toBands()
print(rain_bands.getInfo())
print(rain_bands.bandNames().getInfo())

names_from = rain_bands.bandNames().getInfo()
names_to = ["{year}".format(year=i) for i in range(start_year, end_year + 1)]

rain_bands = rain_bands.select(names_from).rename(names_to)
print(rain_bands.bandNames().getInfo())
print(rain_bands.getInfo())

## Export data - annual sum

In [ ]:
# Get the list of band names
band_names = rain_bands.bandNames().getInfo()

# Loop through the bands and export each one as a separate image
for i in band_names:
    # Select the band
    band = rain_bands.select(i)
    output_name = f"{layer_name}_sum_{aoi_name}_{i}"
    # Define the export parameters
    export_params = {
        "image": band,
        "description": "EXPORT IMAGE TO DRIVE",
        "folder": "GEE_exports",
        "fileNamePrefix": output_name,
        "scale": 500,
        "maxPixels": 1e13,
        "fileFormat": "GeoTIFF",
        "region": aoi.geometry().bounds(),
        "crs": "EPSG:4326"
    }

    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    print("STARTED TASK ", i)

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)